In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using HiLQR

In [ ]:
"""
Exponential spring normal force profile
"""

function ϕbb(x::Vector)::Real
    return x[2]
end

function get_fn_soft(ϕ::Function, σ::Real, ρ::Real)
    fn = x -> σ * exp(-ρ * ϕ(x)) * ϕ(x)
    return fn
end

function get_soft_bb_flow(bb_flow::ControlAffineFlow, fn_soft::Function)
    soft_bb_flow = (x,u) -> bb_flow(x,u) + [zeros(3); fn_soft(x)]
end

In [ ]:
"""
Solver Setup
"""

# Bouncing ball with thrust model
system = bouncing_ball(1.0, 1.0, 9.81)

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.04

# Soft normal force profile
σ = 1
ρ = 1
fn_soft = get_fn_soft(ϕbb, σ, ρ)
nominal_flow = system.modes[:flight].flow
soft_flow = get_soft_bb_flow(nominal_flow, fn_soft)

# Stage and terminal costs
Q = 1e-6 * diagm([1.0, 1.0, 0.0, 0.0])
R = 1e-4 * I
S = 1e+1 * I
Qf = 1e6 * Q

stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x
function contact_stage(x, u)::Real
    contact_err = rk4(nominal_flow, x, u, Δt) - rk4(soft_flow, x, u, Δt)
    #contact_err = nominal_flow(x,u) - soft_flow(x,u)
    contact_cost = contact_err' * S * contact_err
    return stage(x,u) + contact_cost
end

params = HiLQR.Parameters(system, contact_stage, terminal, rk4, N, Δt)

# Reference trajectory and initial conditions
xref = [10.0; 4.0; 0.0; 0.0]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]
params.x0 = [0.0, 4.0, 5.0, 0.0]
params.mI = :flight

nothing

In [ ]:
"""
Solve using HiLQR
"""

# Solve
sol = HiLQR.Solution(params)
sol.us = [10*ones(system.nu) for k = 1:(N-1)]
cache = HiLQR.Cache(params)
@time HiLQR.solve!(sol, cache, params; multishoot=false, αmax=1.0)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,2), xs)

true_cost = HiLQR.TrajectoryCost(stage, terminal)
@show true_cost(params.xrefs, params.urefs, sol.xs, sol.us)

nothing